In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Task 1: Select

Давайте начнем с самого простого. В этом упражнении вам необходимо получить отфильтрованные данные из таблицы в базе данных. Почему важно фильтровать данные именно в запросе, а не после этого в библиотеке Pandas? Потому что таблицы могут иметь огромный размер. Если вы попытаетесь получить всю таблицу, то не сможете ее "переварить" – у вас может не хватить оперативной памяти. Всегда помните об этом.

Первый способ фильтрации — выбрать только те столбцы, которые вам действительно нужны. Второй способ — выбрать только те строки, которые вам действительно нужны.


- [Руководство по SQLite в Python](https://pythonru.com/osnovy/sqlite-v-python)
- [pandas.read_sql](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)
- [pandas.read_sql_query](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)

## 1. Поместите базу данных в подкаталог data в корневом каталоге, используемом в рамках этого проекта.

## 2. Создайте соединение с базой данных с помощью библиотеки sqlite3.

In [3]:
import sqlite3

In [4]:
con = sqlite3.connect('/content/drive/MyDrive/School21/day11/data/checking-logs.sqlite')

## 3. Получите схему таблицы pageviews с помощью pd.io.sql.read_sql и запроса PRAGMA table_info(pageviews)

In [5]:
pd.io.sql.read_sql("PRAGMA table_info('pageviews')", con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


## 4. Получите только первые 10 строк таблицы pageviews, чтобы проверить, как она выглядит

In [6]:
pd.io.sql.read_sql("SELECT * from pageviews LIMIT 10", con)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


## 5. Получите подтаблицу с помощью только одного запроса, в котором:

* используются только uid и datetime;
* используются только данные пользователей (user_*), и не используются данные администраторов;
* сортировка выполняется по uid в порядке возрастания;
* столбец индекса — это datetime;

* datetime преобразован в DatetimeIndex;
* имя датафрейма — pageviews.

In [7]:
pageviews = pd.io.sql.read_sql("""
            SELECT uid, datetime
            FROM pageviews
            WHERE uid LIKE 'user_%' and uid NOT LIKE 'admin_%'
            ORDER BY uid
                      """, con, index_col = 'datetime', parse_dates = ['datetime', 'DatetimeIndex'])

In [8]:
pageviews

,uid
datetime,
2020-04-26 21:53:59.624136,user_1
2020-04-26 22:06:19.478143,user_1
2020-04-26 22:12:09.614497,user_1
2020-04-30 19:29:01.831635,user_1
2020-05-05 20:26:32.894852,user_1
...,...
2020-04-29 16:51:21.877630,user_30
2020-05-09 20:30:47.034282,user_30
2020-05-22 11:30:18.368990,user_5


## 6. Закройте соединение с базой данных.

In [9]:
con.close()